# Quick, Draw!

#### Divya Sanathkumar




The Quick Draw dataset is a collection of images hand drawn by users which consists of about 340 classes. The goal is to build a Neural network that tries to classify these hand drawn images. In order to do that, a Convolutional Neural Networks, used to analyze visual imagery, is built. The data is split into training and validation sets and is fed to this CNN model to predict the accuracy of the image recognition.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from tqdm import tqdm
from PIL import Image

from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential
from keras.layers import *

import tensorflow as tf
from tensorflow.keras.models import Model

# Load and Prep Data

In [ ]:
candle_data = pd.read_csv('../input/quickdraw-doodle-recognition/train_simplified/candle.csv')
candle_data.head()

In [ ]:
strokes_str = candle_data.drawing[50]
print(type(strokes_str))
print(strokes_str)

In [ ]:
strokes_list = eval(strokes_str)
print(type(strokes_list))
print(len(strokes_list))

for s in strokes_list:
    print(s)

## Label Encoder

In [ ]:
path = os.listdir('../input/quickdraw-doodle-recognition/train_simplified')
uniq_labels = np.array(sorted([x[:-4] for x in path]))

In [ ]:
print(len(uniq_labels))
print(uniq_labels[:20])

In [ ]:
enc = LabelEncoder()
enc.fit(uniq_labels)

temp = uniq_labels[[0, 37, 42]] 
print(temp)
print(enc.transform(temp))

In [ ]:
label_lookup = pd.DataFrame({
    'label' : list(map(lambda x : x.replace(' ', '_'), uniq_labels)),
})

label_lookup.head()

In [ ]:
label_lookup.to_csv('path' + 'label_lookup.csv', header=True, index=False)

## Batch Size and Steps Per Epoch

In [ ]:
n_train = 24854214
n_valid = 24854043

print(n_train)
print(n_valid)

In [ ]:
bs = 64

train_steps = 100
valid_steps = 100

print(train_steps)
print(valid_steps)

# Training and Validation Sets

In [ ]:
train_temp = pd.read_csv('../input/quickdrawcombined/train.csv', chunksize=bs)
next(train_temp)

In [ ]:
train = pd.read_csv('../input/quickdrawcombined/train.csv', chunksize=bs)
valid = pd.read_csv('../input/quickdrawcombined/valid.csv', chunksize=bs)

# Data Generators

## img_to_np Function

In [ ]:
def img_to_np(img_str, ht, wt, lw, pad):
    if img_str == 'drawing':
        print(np.zeros((ht, wt), np.uint8))
    
    strokes = eval(img_str)
    
    ht_ = ht - 2*pad
    wt_ = wt - 2*pad
    
    img = np.zeros((ht, wt), np.uint8)

    for s in strokes:
        sx = (np.array(s[0]) * wt_ / 256).round().astype('int') + pad
        sy = (np.array(s[1]) * ht_ / 256).round().astype('int') + pad
        for i in range(len(sx) - 1):
            p1 = (sx[i],   sy[i])
            p2 = (sx[i+1], sy[i+1])
            img = cv2.line(img, p1, p2, (255, 0, 0), lw, lineType=cv2.LINE_AA)
            
    return img

In [ ]:
img_array = img_to_np(strokes_str, 64,64,1,5)

plt.imshow(img_array, cmap='binary')
plt.axis('off')
plt.show()

## Create Generator

In [ ]:
class DataGenerator(keras.utils.Sequence):
    
    #####################################################################
    # Constructor
    # - df is a TextFileReader for reading in DataFrame
    #####################################################################
    def __init__(self, df, n_classes, batch_size, n_steps, img_params):
        #self.path = path
        self.df = df
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.img_params = img_params
        
    #####################################################################
    # __getitem__ 
    # This is directly called by Keras methods. It returns a single 
    # batch of data. 
    #####################################################################
    def __getitem__(self, index):
        
        # Typically, this would determine the rows to select for the
        # current batch. In our case, we will simply grab the next 
        # batch from the TextFileReader

        X, y = self.__data_generation(index)

        return X, y

    #####################################################################
    # __data_generation 
    # This is called by __getitem__ to generate the batch.
    #####################################################################
    def __data_generation(self, index):

        # Get next batch
        batch = next(self.df)

        # Create blank canvas
        ht, wt, lw, pad = self.img_params
        X = np.zeros(shape=(len(batch), ht, wt, 1))
        
        ###########################################################
        #print(index, len(batch), batch.columns)


        # Process each image in the batch
        for i, img_str in enumerate(batch.drawing.values):

            if img_str == 'drawing':
                img_str == batch.drawing.values[i+1]
                batch.word.values[i] = batch.word.values[i+1]

            X[i, :, :, 0] = img_to_np(img_str, ht, wt, lw, pad) / 255

        # Get batch labels
        labels = batch.word.values
        y = enc.transform(labels)

        return X, y

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.n_steps

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return None

## Display Batch Images

In [ ]:
temp_dg = DataGenerator(train_temp, n_classes=20, batch_size=bs, n_steps=10,
                        img_params=(64, 64, 1, 2))

X, y = temp_dg.__getitem__(2)

labels = uniq_labels[y]

plt.figure(figsize=[12,12])
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(X[i,:,:,0], cmap='binary')
    plt.title(labels[i])
    plt.axis('off')
plt.tight_layout()
plt.show()

## Create Train and Valid Generators

In [ ]:
train_dg = DataGenerator(train, batch_size=bs, n_classes=20, n_steps=train_steps,
                         img_params=(64, 64, 1, 2))

valid_dg = DataGenerator(valid, batch_size=bs, n_classes=20, n_steps=valid_steps,
                         img_params=(64, 64, 1, 2))

# CNN

## Build Network

In [ ]:
np.random.seed(1)

cnn = Sequential()

cnn.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', input_shape=(64,64,1)))
cnn.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
cnn.add(MaxPooling2D(2,2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
cnn.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
cnn.add(MaxPooling2D(2,2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same'))
cnn.add(Conv2D(512, (3,3), activation = 'relu', padding = 'same'))
cnn.add(MaxPooling2D(2,2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(1024, (3,3), activation = 'relu', padding = 'same'))
cnn.add(Conv2D(1024, (3,3), activation = 'relu', padding = 'same'))
cnn.add(MaxPooling2D(2,2))
cnn.add(BatchNormalization())
cnn.add(Flatten())

cnn.add(Dense(2048, activation='relu'))
cnn.add(BatchNormalization())

cnn.add(Dense(1024, activation='relu'))
cnn.add(BatchNormalization())

cnn.add(Dense(512, activation='relu'))
cnn.add(BatchNormalization())

cnn.add(Dense(340, activation='softmax'))

In [ ]:
cnn.summary()

## Train Network

### Run 1

In [ ]:
%%time 

opt = keras.optimizers.Adam(0.001)
cnn.compile(loss='SparseCategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])

h1 = cnn.fit(train_dg, validation_data=valid_dg,
             verbose=1, epochs=100, batch_size = 1000)

## Create Function to display plot

In [ ]:
def vis_training(hlist, start=1, size=[12,6], show_val=True):
    tr_loss = []
    va_loss = []
    tr_acc = []
    va_acc = []
    for h in hlist:
        tr_loss += h.history['loss'] 
        va_loss += h.history['val_loss']
        tr_acc += h.history['accuracy'] 
        va_acc += h.history['val_accuracy']
    
    plt.figure(figsize = size)
    a = start
    b = len(tr_loss) + 1
    plt.subplot(1,2,1)
    plt.plot(range(a,b), tr_loss[a-1:], label='Training')
    if(show_val): 
        plt.plot(range(a,b), va_loss[a-1:], label='Validation')
    plt.title('Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(range(a,b), tr_acc[a-1:], label='Training')
    if(show_val):  
        plt.plot(range(a,b), va_acc[a-1:], label='Validation')
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
vis_training([h1])

### Run 2

In [ ]:
h2 = cnn.fit(train_dg, validation_data=valid_dg,
             verbose=1, epochs=100,batch_size=100)

In [ ]:
vis_training([h1,h2])

### Run 3

In [ ]:
keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)
h3 = cnn.fit(train_dg, validation_data=valid_dg,
             verbose=1, epochs=100,batch_size=100)

In [ ]:
vis_training([h1, h2, h3])

# Evaluate the Model

## Distribution of Top 3 Probabilities

In [ ]:
def get_top_3(probs):
    top_classes = np.argpartition(probs, -3)[-3:]                  # Gives top 3 classes in increasing order
    top_classes = top_classes[np.argsort(probs[top_classes])]      # Sorts in increasing order
    top_classes = np.flip(top_classes)                             # Flips the order.
    top_probs = probs[top_classes]              

    return top_probs, top_classes

In [ ]:
NB = len(train_dg)
#NB = 10

top_3_probs = np.zeros(shape=(64*NB, 3))

for i in range(NB):
    batch_imgs, batch_labels = train_dg.__getitem__(i)
    batch_pred = cnn.predict(batch_imgs)

    ## Loop over each image in the batch
    for j in range(64):
        top_probs, top_classes = get_top_3(batch_pred[j, :])
        top_3_probs[i,:] = top_probs

print(top_3_probs.shape)

plt.figure(figsize=[10,6])
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.hist(top_3_probs[:,i], color='orchid', edgecolor='k', bins = np.arange(0, 1.01, 0.025))
    plt.yscale('log')
plt.show()

# MAP at 3

In [ ]:
def MAP3(t):
    NB = len(valid_dg)
    #NB = 20

    sum_ap3 = 0
    N_obs = 0

    for i in range(NB):
        batch_imgs, batch_labels = train_dg.__getitem__(i)
        batch_pred = cnn.predict(batch_imgs)

        ## Loop over each image in the batch
        for j in range(64):
            probs = batch_pred[j, :]
            top_classes = np.argpartition(probs, -3)[-3:]                  # Gives top 3 classes in increasing order
            top_classes = top_classes[np.argsort(probs[top_classes])]      # Sorts in increasing order
            top_classes = np.flip(top_classes)                             # Flips the order.

            top_probs = probs[top_classes]              # Don't need this when not using a threshold

            # Keep Probs Over Threshold
            sel = top_probs > t
            sel[0] = True                               # Always keep first pred
            top_classes = top_classes[sel]

            K = len(top_classes)   # Number of classes to submit
            if K == 3:
                scores = np.array([11/18, 5/18, 2/18])
            elif K == 2:
                scores = np.array([3/4, 1/4])
            else:
                scores = np.array([1])
            
            sel = (top_classes == batch_labels[j])
            ap3 = np.sum(scores * sel)

            sum_ap3 += ap3
            N_obs += 1
            
            #print(ap3)

    map3 = sum_ap3 / N_obs

    return map3

for t in np.arange(0, 1.01, 0.05):
    print(round(t, 2), '\t', MAP3(t))

In [ ]:
%%time

MAP3_scores = []
t_array = np.arange(0.05, 1.01, 0.05) 

for t in t_array:
    MAP3_scores.append(MAP3(t))

plt.plot(t_array, MAP3_scores)
plt.scatter(t_array, MAP3_scores)
plt.show()

# CAM

## Create CAM Function

In [ ]:
class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName
        if self.layerName is None:
            self.layerName = self.find_target_layer()
            
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")
        
    def compute_heatmap(self, image, eps=1e-8):
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output,self.model.output]
       )
           
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]
            grads = tape.gradient(loss, convOutputs)

            castConvOutputs = tf.cast(convOutputs > 0, "float32")
            castGrads = tf.cast(grads > 0, "float32")
            guidedGrads = castConvOutputs * castGrads * grads
            convOutputs = convOutputs[0]
            guidedGrads = guidedGrads[0]

            weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
            cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

            (w, h) = (image.shape[2], image.shape[1])
            heatmap = cv2.resize(cam.numpy(), (w, h))
            numer = heatmap - np.min(heatmap)
            denom = (heatmap.max() - heatmap.min()) + eps
            heatmap = numer / denom
            heatmap = (heatmap * 255).astype("uint8")
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap = cv2.COLORMAP_VIRIDIS):
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        return (heatmap, output)

In [ ]:
train_new = pd.read_csv('../input/quickdrawcombined/train.csv', chunksize=bs)

train_dg = DataGenerator(train_new, batch_size=bs, n_classes=20, n_steps=train_steps,
                         img_params=(64, 64, 1, 2))

X, y = train_dg.__getitem__(2)

labels = uniq_labels[y]

plt.figure(figsize=[12,12])
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(X[i,:,:,0], cmap='binary')
    plt.title(labels[i])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
batch_pred = cnn.predict(X)
batch_pred.shape

In [ ]:
plt.figure(figsize=[16,48])
for n in range(64):

    top_probs, top_classes = get_top_3(batch_pred[n, :])
    
    cam = GradCAM(cnn, top_classes[0])             
    heatmap = cam.compute_heatmap(X[[n], :, :, :]) 

    plt.subplot(16,4,n+1)
    plt.imshow(X[n,:,:,0], cmap='binary')
    plt.imshow(heatmap, alpha=0.6, cmap='coolwarm')
    plt.title(f'{labels[n]} - {uniq_labels[top_classes]} \n{top_classes} - {top_probs.round(2)}')
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Save the Model

In [ ]:
cnn.save('demo_model.h5')

# Load Test Data

In [ ]:
test = pd.read_csv('../input/quickdraw-doodle-recognition/test_simplified.csv')
print(test.shape)
test.head()

## Convert Test Strings to Arrays

In [ ]:
def img_to_np(img_str, ht, wt, lw, pad):

    strokes = eval(img_str)

    ht_ = ht - 2*pad
    wt_ = wt - 2*pad

    img = np.zeros((ht, wt), np.uint8)

    for s in strokes:
        sx = (np.array(s[0]) * wt_ / 256).round().astype('int') + pad
        sy = (np.array(s[1]) * ht_ / 256).round().astype('int') + pad

        for i in range(len(sx) - 1):
            p1 = (sx[i],   sy[i])
            p2 = (sx[i+1], sy[i+1])
            img = cv2.line(img, p1, p2, (255, 0, 0), lw, lineType=cv2.LINE_AA)
            #img = cv2.resize(img, (ht, wt))
    return img

In [ ]:
test_imgs = np.zeros(shape = (test.shape[0], 64, 64, 1))

In [ ]:
%%time

for i, row in test.iterrows():
    test_imgs[i,:,:,0] = img_to_np(row.drawing, 64, 64, 1, 2) / 255

In [ ]:
plt.figure(figsize=[12,12])
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(test_imgs[i,:,:,0], cmap='binary')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
sample = test.sample(64)
sample.shape

plt.figure(figsize=[12,12])
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(img_to_np(sample.drawing.values[i], 64, 64, 1, 2), cmap='binary')
    plt.axis('off')
plt.tight_layout()
plt.show()

## Load Model

In [ ]:
cnn = keras.models.load_model('./demo_model.h5')
cnn.summary()

## Generate Predictions

In [ ]:
test_imgs.shape

In [ ]:
%%time 

probs = cnn.predict(test_imgs)

print(probs.shape)

## Distribution of Top 3 Probabilities

In [ ]:
N_train = probs.shape[0]
top_3_probs = np.zeros(shape=(N_train, 3))

for i in range(N_train):
    p = probs[i, :]
    top_classes = np.argpartition(p, -3)[-3:]                      # Gives top 3 classes in increasing order
    top_classes = top_classes[np.argsort(p[top_classes])]      # Sorts in increasing order
    top_classes = np.flip(top_classes)                             # Flips the order.

    top_probs = p[top_classes]              

    top_3_probs[i,:] = top_probs
    
print(top_3_probs[:10, :].round(2))

print(top_3_probs.shape)

plt.figure(figsize=[10,6])
for i in range(3):
    plt.subplot(3,1,i+1)
    plt.hist(top_3_probs[:,i], color='orchid', edgecolor='k', bins = np.arange(0, 1.01, 0.025))
    plt.yscale('log')
plt.show()

## Determine Predictions

In [ ]:
N_train = probs.shape[0]
predictions = []

t = 0.35

for i in range(N_train):
    p = probs[i, :]
    top_classes = np.argpartition(p, -3)[-3:]                   # Gives top 3 classes in increasing order
    top_classes = top_classes[np.argsort(p[top_classes])]       # Sorts in increasing order
    top_classes = np.flip(top_classes)                          # Flips the order.

    top_probs = p[top_classes]              

    # Keep Probs Over Threshold
    sel = top_probs > t
    sel[0] = True                               # Always keep first pred
    predictions.append(top_classes[sel])
    
print(len(predictions))

# Create Submission

In [ ]:
test.head()

In [ ]:
submission = pd.read_csv('../input/quickdraw-doodle-recognition/sample_submission.csv')
submission.head()

In [ ]:
len(submission)

In [ ]:
label_lookup_df = pd.read_csv('path' + 'label_lookup.csv')
label_lookup = {k:v for k,v in zip(label_lookup_df.index.values, label_lookup_df.label.values)}
label_lookup[0]

In [ ]:
%%time

for i in range(N_train):
    classes = predictions[i]
    words_list = [label_lookup[c] for c in classes]
    words_string = ' '.join(words_list)
    submission.loc[i, 'word'] = words_string
    #print(words_string)
    
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

# Images with Predictions

In [ ]:
idx = np.random.choice(range(N_train), 64, replace=False)
test_sample = test.iloc[idx,:]
sub_sample = submission.iloc[idx, :]

plt.figure(figsize=[16,16])

for i in range(64):
    plt.subplot(8,8,i+1)
    plt.imshow(img_to_np(test_sample.drawing.values[i], 64, 64, 1, 2), cmap='binary')
    plt.title(sub_sample.word.values[i].replace(' ', '\n'))
    plt.axis('off')
plt.tight_layout()
plt.show()